In [77]:
import os 
import pyttsx3
from ebooklib import epub
from bs4 import BeautifulSoup
import fitz
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import threading
import json


In [78]:
# Initialize pyttsx3 engine
engine = pyttsx3.init()

In [79]:
# Set default properties
engine.setProperty('rate', 150)    # Speech rate
engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)

In [80]:
# Global variable to hold the current text
current_text = ""

In [81]:
voices = engine.getProperty('voices')

for index, voice in enumerate(voices):
    print(f"Voice {index}: {voice.name} - {voice.languages}")

Voice 0: Microsoft David Desktop - English (United States) - []
Voice 1: Microsoft Zira Desktop - English (United States) - []


In [82]:
## Function to read .txt files
def read_text_file(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        return file.read()

In [83]:
## Function to read .epub files
def read_epub_file(file_path):
    book = epub.read_pub(file_path)
    text = "" 

    for doc in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        content = doc.get_content().decode('utf-8')
        soup = BeautifulSoup(content, 'html.parser')
        text += soup.get_text()

    return text

In [84]:
# Function to read .pdf files
def read_pdf_file(file_path):
    pdf = fitz.open(file_path)
    text = ""

    for page in pdf:
        text += page.get_text()

    return text

In [85]:
# Playback Control Functions

def play_text():
    global current_text

    if not current_text:
        messagebox.showwarning("No Text Loaded", "Please load a text file first.")
        return
    
    try:
        threading.Thread(target=tts_engine, args=(current_text,), daemon=True).start()

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during playback:\n{e}")

def tts_engine(text):
    engine.say(text)
    engine.runAndWait()


def pause_reading():
    engine.pause()

def resume_reading():
    engine.resume()

def stop_reading():
    engine.stop()

In [86]:
# Speech Settings Functions 

def set_voice(event=None):
    voice_id = voice_combo.current()
    if voice_id >= 0 and voice_id < len(voices):
        engine.setProperty('voice', voices[voice_id].id)

def set_rate(event=None):
    try:
        rate = int(rate_slider.get())
        engine.setProperty('rate', rate)
        
    except ValueError:
        messagebox.showerror("Invalid Rate", "Please enter a valid integer for rate.")

def set_volume(event=None):
    try:
        volume = float(volume_slider.get())
        engine.setProperty('volume', volume)

    except ValueError:
        messagebox.showerror("Invalid Volume", "Please enter a valid float for volume.")

In [87]:
# Function to Save as Audio File
def save_to_audio():
    global current_text

    if not current_text:
        messagebox.showwarning("No Text Loaded", "Please load a text file first.")
        return
    
    file_path = filedialog.asksaveasfilename(defaultextension=".mp3",
                                             filetypes=[("MP3 Files", "*.mp3"), ("WAV Files", "*.wav")])
    if file_path:
        try:
            engine.save_to_file(current_text, file_path)
            engine.runAndWait()
            messagebox.showinfo("Success", f"Audio saved successfully at {file_path}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save audio:\n{e}")

In [88]:
# Function to Load File
def load_file():
    global current_text

    file_path = filedialog.askopenfilename(filetypes=[
        ("Supported Files", "*.txt *.TXT *.epub *.EPUB *.pdf *.PDF"),
        ("Text Files", "*.txt *.TXT"),
        ("EPUB Files", "*.epub *.EPUB"),
        ("PDF Files", "*.pdf *.PDF"),
        ("All Files", "*.*")
    ])

    if file_path:
        try:
            if file_path.endswith('.txt'):
                current_text = read_text_file(file_path)
            elif file_path.endswith('.epub'):
                current_text = read_epub_file(file_path)
            elif file_path.endswith('.pdf'):
                current_text = read_pdf_file(file_path)
            else:
                messagebox.showerror("Unsupported File", "Please select a .txt, .epub, or .pdf file.")
                return
            
            text_display.delete(1.0, tk.END)
            text_display.insert(tk.END, current_text)
            messagebox.showinfo("File Loaded", f"Successfully loaded {os.path.basename(file_path)}")

        except Exception as e:
            messagebox.showerror("Error", f"Failed to load file:\n{e}")

In [ ]:
# Initialize Tkinter
root = tk.Tk()
root.title("Text-to-Speech Application")
root.geometry("800x600")

# Create Frames for better layout management
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

middle_frame = tk.Frame(root)
middle_frame.pack(pady=10)

bottom_frame = tk.Frame(root)
bottom_frame.pack(pady=10)

# Load File Button
load_button = tk.Button(top_frame, text="Load File", command=load_file, width=20)
load_button.pack()

# Text Display Area with Scrollbar
text_display = tk.Text(middle_frame, wrap=tk.WORD, width=80, height=20)
text_display.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(middle_frame, command=text_display.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

text_display.config(yscrollcommand=scrollbar.set)

# Playback Control Buttons
play_button = tk.Button(bottom_frame, text="Play", command=play_text, width=10)
play_button.grid(row=0, column=0, padx=5, pady=5)

pause_button = tk.Button(bottom_frame, text="Pause", command=pause_reading, width=10)
pause_button.grid(row=0, column=1, padx=5, pady=5)

resume_button = tk.Button(bottom_frame, text="Resume", command=resume_reading, width=10)
resume_button.grid(row=0, column=2, padx=5, pady=5)

stop_button = tk.Button(bottom_frame, text="Stop", command=stop_reading, width=10)
stop_button.grid(row=0, column=3, padx=5, pady=5)

save_button = tk.Button(bottom_frame, text="Save as Audio", command=save_to_audio, width=15)
save_button.grid(row=0, column=4, padx=5, pady=5)

# Speech Settings
settings_frame = tk.LabelFrame(root, text="Speech Settings")
settings_frame.pack(pady=10, padx=10, fill="x")

# Voice Selection
tk.Label(settings_frame, text="Voice:").grid(row=0, column=0, padx=5, pady=5, sticky='e')
voice_combo = ttk.Combobox(settings_frame, state="readonly")
voice_names = [f"{voice.name} ({', '.join(voice.languages)})" for voice in voices]
voice_combo['values'] = voice_names
voice_combo.current(0)  # Set default voice
voice_combo.grid(row=0, column=1, padx=5, pady=5, sticky='w')
voice_combo.bind("<<ComboboxSelected>>", set_voice)

# Speech Rate
tk.Label(settings_frame, text="Rate:").grid(row=1, column=0, padx=5, pady=5, sticky='e')
rate_slider = tk.Scale(settings_frame, from_=100, to=200, orient=tk.HORIZONTAL, command=lambda val: set_rate())
rate_slider.set(engine.getProperty('rate'))
rate_slider.grid(row=1, column=1, padx=5, pady=5, sticky='w')

# Volume Control
tk.Label(settings_frame, text="Volume:").grid(row=2, column=0, padx=5, pady=5, sticky='e')
volume_slider = tk.Scale(settings_frame, from_=0.0, to=1.0, resolution=0.1, orient=tk.HORIZONTAL, command=lambda val: set_volume())
volume_slider.set(engine.getProperty('volume'))
volume_slider.grid(row=2, column=1, padx=5, pady=5, sticky='w')

# Start the Tkinter event loop
root.mainloop()

Exception in thread Thread-11 (tts_engine):
Traceback (most recent call last):
  File "c:\Users\asust\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\asust\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\asust\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\asust\AppData\Local\Temp\ipykernel_20392\3270805405.py", line 18, in tts_engine
  File "c:\Users\asust\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\asust\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    ret